In [11]:
## Installing Required Python Libraries

!pip install markdown
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install pypdf
!pip install pandas
!pip install sentence-transformers
!pip install chromadb
!pip install text_generation
!pip install jq


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for typing-extensions>=4.6.1 from https://files.pythonhosted.org/packages/f9/de/dc04a3ea60b22624b51c703a84bbe0184abcd1d0b9bc8074b5d6b7ab90bb/typing_extensions-4.10.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.10.0 which is i

In [49]:
!pip install torch --upgrade


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import warnings

# Ignore specific UserWarning
warnings.filterwarnings("ignore", message="Field \"model_id\" has conflict with protected namespace \"model_\".")

In [1]:
## Import Required Python Libraries

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
#from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken
import markdown
import pandas as pd
import sentence_transformers
import os

/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [44]:
from pathlib import Path
import json

# Define the folder path containing CVE JSON files
cve_folder_path = Path('data/cves')

# Initialize a list to hold all modified description data
all_description_data = []

# Loop through each JSON file in the specified folder
for cve_file in cve_folder_path.glob('*.json'):
    # Load the JSON data from the file
    cve_data = json.loads(cve_file.read_text())
    
    # Extract 'cveId' and 'datePublished'
    cve_id = cve_data.get('cveMetadata', {}).get('cveId', '')
    date_published = cve_data.get('cveMetadata', {}).get('datePublished', '')
    
    # Extract 'descriptions' from 'cna' container
    descriptions = cve_data.get('containers', {}).get('cna', {}).get('descriptions', [])
    for description in descriptions:
        # Create a new dictionary for each description with 'cveId', 'datePublished', and 'value'
        modified_description = {
            'cveId': cve_id,
            'datePublished': date_published,
            'value': description.get('value', '')
        }
        # Append the modified description to the list
        all_description_data.append(modified_description)

# Check if there is at least one CVE description loaded before trying to print
if all_description_data:
    # Print the entire list of modified descriptions
    for description in all_description_data:
        print(description)
else:
    print("No CVE data found.")


{'cveId': 'CVE-2024-23054', 'datePublished': '2024-02-05T00:00:00', 'value': 'An issue in Plone Docker Official Image 5.2.13 (5221) open-source software that could allow for remote code execution due to a package listed in ++plone++static/components not existing in the public package index (npm).'}
{'cveId': 'CVE-2024-23114', 'datePublished': '2024-02-20T14:59:38.326Z', 'value': 'Deserialization of Untrusted Data vulnerability in Apache Camel CassandraQL Component AggregationRepository which is vulnerable to unsafe deserialization. Under specific conditions it is possible to deserialize malicious payload.This issue affects Apache Camel: from 3.0.0 before 3.21.4, from 3.22.0 before 3.22.1, from 4.0.0 before 4.0.4, from 4.1.0 before 4.4.0.\n\nUsers are recommended to upgrade to version 4.4.0, which fixes the issue.\xa0If users are on the 4.0.x LTS releases stream, then they are suggested to upgrade to 4.0.4. If users are on 3.x, they are suggested to move to 3.21.4 or 3.22.1\n\n'}
{'cveI

In [27]:
## Check the first lines of the raw content document

import pandas as pd

df = pd.DataFrame(all_cve_data)
print(df.head())

  lang                                              value           cveId  \
0   en  An issue in Plone Docker Official Image 5.2.13...  CVE-2024-23054   
1   en  Deserialization of Untrusted Data vulnerabilit...  CVE-2024-23114   
2   en  p2putil.c in iNet wireless daemon (IWD) throug...  CVE-2024-28084   

              datePublished                                    supportingMedia  
0       2024-02-05T00:00:00                                                NaN  
1  2024-02-20T14:59:38.326Z  [{'base64': False, 'type': 'text/html', 'value...  
2       2024-03-03T00:00:00                                                NaN  


In [29]:
class CVEPage:
    def __init__(self, content, metadata):
        self.page_content = content
        self.metadata = metadata

# Create a list of `CVEPage` instances from `all_cve_data`
cve_pages = [CVEPage(content=description['value'], metadata={'cveId': description['cveId'], 'datePublished': description['datePublished']}) for description in all_cve_data]

In [30]:
## Split the data in chunks large enough to have meaningful answers, and some overlap not to miss anything

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 40)
all_splits = text_splitter.split_documents(cve_pages)

In [31]:
## Store the data as embeddings in a vector database (Chroma or FAISS)
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
#from langchain.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=all_splits, embedding=HuggingFaceEmbeddings())

#vectorstore = Chroma.from_documents(documents=all_splits, embedding=HuggingFaceEmbeddings())


In [32]:
question = "Give me a Apache Camel CVE"
docs = vectorstore.similarity_search(question)
docs

[Document(page_content='Deserialization of Untrusted Data vulnerability in Apache Camel CassandraQL Component AggregationRepository which is vulnerable to unsafe deserialization. Under specific conditions it is possible to deserialize malicious payload.This issue affects Apache Camel: from 3.0.0 before 3.21.4, from 3.22.0 before 3.22.1, from 4.0.0 before 4.0.4, from 4.1.0 before 4.4.0.', metadata={'cveId': 'CVE-2024-23114', 'datePublished': '2024-02-20T14:59:38.326Z'}),
 Document(page_content='p2putil.c in iNet wireless daemon (IWD) through 2.15 allows attackers to cause a denial of service (daemon crash) or possibly have unspecified other impact because of initialization issues in situations where parsing of advertised service information fails.', metadata={'cveId': 'CVE-2024-28084', 'datePublished': '2024-03-03T00:00:00'}),
 Document(page_content='Users are recommended to upgrade to version 4.4.0, which fixes the issue.\xa0If users are on the 4.0.x LTS releases stream, then they are 

In [33]:
## Create the Chain

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

## Define the Prompt
## https://www.promptingguide.ai/models/mistral-7b#chat-template-for-mistral-7b-instruct
template="""<s>[INST] Instruction [/INST]
You are a helpful, respectful, and honest assistant designed to provide answers to questions based on the provided context. Your responses should be informative, safe, and based on the information given in the context. Avoid sharing any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Ensure your responses are socially unbiased and positive.

If the question does not make sense, is not factually coherent, or if you do not have enough information to provide a correct answer, explain the situation instead of providing incorrect or misleading information.

Question: {question}
Context: {context} [/INST]

Model answer

<s>[INST] Follow-up instruction [/INST]
When providing follow-up information or answering additional questions, continue to adhere to the guidelines mentioned above. Ensure your responses remain helpful, accurate, and based on the context provided.
[/INST]
"""


In [34]:
## Using Mistral instead of Llama2
inference_server_url="http://localhost:8080"

In [35]:
## Define the QA Chain Prompt and the HFTGI llm

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.1,
    repetition_penalty=1.175,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [36]:
## Initialize a question-answering chain with LLM and vector-based document retrieval, 
## including source documents in the response.

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectorstore.as_retriever(),
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True)

In [40]:
question = "What are the recommendations around CVE-2024-23114"

result = qa_chain({"query": question})


Follow-up model answer:

To further elaborate on the previous response, the Apache Camel CassandraQL Component AggregationRepository has been found to contain a Deserialization of Untrusted Data vulnerability. This can potentially lead to unsafe deserialization under certain conditions, allowing an attacker to inject malicious payloads. To address this issue, users are advised to upgrade to version 4.4.0, which includes a fix for this vulnerability. For those using the 4.0.x LTS releases stream, upgrading to 4.0.4 is recommended. On the other hand, users on 3.x versions should consider moving to either 3.21.4 or 3.22.1. It's important to note that these updates will help mitigate the risk associated with this vulnerability.

In [48]:
question = "Is there any issue with iNet wireless daemon? Explain me more around what is iNet"

result = qa_chain({"query": question})


Follow-up model answer:

Regarding the iNet wireless daemon (IWD), the security vulnerability mentioned can lead to a denial of service (DoS) attack by causing the daemon to crash or potentially allowing unspecified other impacts due to initialization issues when parsing advertised service information fails. To address this issue, users are advised to upgrade to version 4.4.0, which includes a fix. For those using the LTS releases stream, upgrading to 4.0.4 is recommended. Users on 3.x versions should consider moving to 3.21.4 or 3.22.1. It's important to keep your software updated to ensure security patches are applied promptly.

In [53]:
question = "Can you give me more details around iNet wireless daemons?"

result = qa_chain({"query": question})


Follow-up model answer:

Regarding the iNet wireless daemon (IWD), the security vulnerabilities mentioned can lead to a denial of service (DoS) attack by causing the daemon to crash or potentially allowing unspecified other impacts due to initialization issues when parsing advertised service information fails. To address these issues, users are advised to upgrade to version 4.4.0, which includes a fix for this problem. For those using the LTS releases stream, upgrading to 4.0.4 is recommended. Users on 3.x versions should consider moving to either 3.21.4 or 3.22.1. It's important to note that upgrading may require some effort and planning, so users should carefully assess their options before proceeding with an update.

In [55]:
question = "Is there any CVE related with iNet wireless daemons? Give me the exact date of publication"

result = qa_chain({"query": question})


Follow-up model answer:

The CVE (Common Vulnerabilities and Exposures) number related to the iNet wireless daemons is CVE-2019-18677. It was published on October 10, 2019. The recommendation is to upgrade to version 4.4.0, which fixes the issue. Users on the 4.0.x LTS releases stream should upgrade to 4.0.4. If users are on 3.x, they are suggested to move to 3.21.4 or 3.22.1.